In [13]:
# 导入机器学习常用的包
import polars as pl
import numpy as np
import pandas as pd
# 导入逻辑回归分类常用的包

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
path = 'data/'
n_rows=100

In [ ]:
# 数据预处理
class Pipeline:
    @staticmethod
    def set_table_dtypes(df):
        dtype_mapping = {
            "case_id": pl.Int32,
            "WEEK_NUM": pl.Int32,
            "num_group1": pl.Int32,
            "num_group2": pl.Int32,
            "date_decision": pl.Date,
        }
        for col in df.columns:
            if col in dtype_mapping:
                df = df.with_columns(pl.col(col).cast(dtype_mapping[col]))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col[-1] == "M":
                df = df.with_columns(pl.col(col).cast(pl.String))
            elif col[-1] == "D":
                df = df.with_columns(pl.col(col).cast(pl.Date))
        return df

    @staticmethod
    def handle_dates(df):
        # 确保 date_decision 列是日期类型
        if df['date_decision'].dtype == pl.String:
            df = df.with_columns(
                pl.col('date_decision').str.strptime(pl.Date, format='%Y-%m-%d', strict=False)
            )
        
        # 处理可能存在的无效日期
        df = df.filter(pl.col('date_decision').is_not_null())

        # 获取所有以 'D' 结尾的列
        date_cols = [col for col in df.columns if col[-1] == "D"]
        
        for col in date_cols:
            # 确保所有日期列是日期类型
            if df[col].dtype == pl.String:
                df = df.with_columns(
                    pl.col(col).str.strptime(pl.Date, format='%Y-%m-%d', strict=False)
                )
            
            # 过滤掉无效的日期
            df = df.filter(pl.col(col).is_not_null())
            
            # 计算与 date_decision 的天数差
            df = df.with_columns(
                (pl.col(col) - pl.col("date_decision")).alias(col)
            ).with_columns(
                pl.col(col).dt.total_days().cast(pl.Float32).alias(col)
            )
        
        # 删除不再需要的列
        df = df.drop(["date_decision", "MONTH"])
        return df
    @staticmethod
    def filter_cols(df):
        # 调试信息：检查初始数据形状
        print("Initial df shape before filtering:", df.shape)

        # 将 None 转换为 NaN，以便后续处理
        df = df.fill_nan(None)

        # 计算每列的空值比例
        null_ratio = df.select([pl.col(col).is_null().mean().alias(col) for col in df.columns])
        high_null_cols = [col for col in null_ratio.columns if null_ratio[col][0] > 0.95 and col not in ["target", "case_id", "WEEK_NUM"]]
        df = df.drop(high_null_cols)

        # 调试信息：检查删除高缺失率列后的数据形状
        print("df shape after dropping high null columns:", df.shape)

        # 处理字符串列
        string_cols = [col for col in df.columns if df[col].dtype == pl.String]
        for col in string_cols:
            if col not in ["target", "case_id", "WEEK_NUM"]:
                freq = df[col].n_unique()
                if freq == 1 or freq > 200:
                    df = df.drop(col)

        # 调试信息：检查删除高唯一值字符串列后的数据形状
        print("df shape after dropping high unique value string columns:", df.shape)

        # 确保所有数值列被正确转换
        for col in df.columns:
            if col not in ["target", "case_id", "WEEK_NUM"] and df[col].dtype in [pl.Float64, pl.Float32, pl.Int32, pl.Int64]:
                df = df.with_columns(pl.col(col).fill_nan(0).cast(pl.Float32))

        # 调试信息：检查最终数据形状
        print("Final df shape after all filters:", df.shape)

        return df

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 创建示例 DataFrame
data = {
    'Feature1': ['A', 'B', 'C', 'A', 'B'],
    'Feature2': [1, 2, 3, 4, 5],
    'Target': [0, 1, 0, 1, 0]
}

df = pd.DataFrame(data)

# 初始化 OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)

# 应用独热编码
encoded_data = one_hot_encoder.fit_transform(df[['Feature1']])
encoded_df = pd.DataFrame(encoded_data, columns=one_hot_encoder.get_feature_names_out(['Feature1']))
df_encoded = pd.concat([df[['Feature2']], encoded_df], axis=1)

# 定义特征和目标变量
X = df_encoded
y = df['Target']

# 分割数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 训练逻辑回归模型
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# 预测
y_pred = logreg.predict(X_test)

# 评估模型
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.00


In [6]:
encoded_data

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [14]:
train_base = pl.read_parquet(path+'parquet_files/train/train_base.parquet')# 其他表要连接到这个训练主表
# test_base = pl.read_parquet(path+'parquet_files/test/test_base.parquet')

In [15]:
train_static = pl.concat(
    [
        pl.read_parquet(path + "parquet_files/train/"+ "train_static_0_0.parquet"),
        pl.read_parquet(path + "parquet_files/train/"+ "train_static_0_1.parquet")
    ]
)

In [16]:
train_static.head()

case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,…,numinstpaidearlyest_4493214L,numinstpaidlastcontr_4325080L,numinstpaidlate1d_3546852L,numinstregularpaid_973L,numinstregularpaidest_4493210L,numinsttopaygr_769L,numinsttopaygrest_4493213L,numinstunpaidmax_3546851L,numinstunpaidmaxest_4493212L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,opencred_647L,paytype1st_925L,paytype_783L,payvacationpostpone_4187118D,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,previouscontdistrict_112M,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,validfrom_1069D
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
0,null,null,1917.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,null,"""OTHER""","""OTHER""",null,null,null,null,null,null,24.0,0.0,0.0,null,"""a55475b1""",null,0.0,0.0,null,null,0.0,0.0,null,"""BO""",null,null
1,null,null,3134.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,null,"""OTHER""","""OTHER""",null,null,null,null,null,null,18.0,0.0,0.0,null,"""a55475b1""",null,0.0,0.0,null,null,0.0,0.0,null,"""BO""",null,null
2,null,null,4937.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,false,"""OTHER""","""OTHER""",null,null,null,null,null,null,36.0,0.0,0.0,null,"""a55475b1""",null,0.0,0.0,null,null,0.0,0.0,null,"""BO""","""AL""",null
3,null,null,4643.6,0.0,0.0,1.0,0.0,2.0,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,1.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,1.0,false,"""OTHER""","""OTHER""",null,null,null,null,null,null,12.0,0.0,0.0,null,"""a55475b1""",null,1.0,1.0,null,null,0.0,0.0,null,"""BO""","""AL""",null
4,null,null,3390.2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,false,"""OTHER""","""OTHER""",null,null,null,null,null,null,24.0,0.0,0.0,null,"""a55475b1""",null,0.0,0.0,null,null,0.0,0.0,null,"""BO""","""AL""",null


In [ ]:
#也可对每张表的列去重
# 每个文件先导进来
# 同一张表先拼接上
# 每种表有什么特征
# 表互相怎么连接
# 把表有必要的部分连接到base表上
# 然后EDA 分析
# 除了主表外，还有9张数据表+两张说明表

# applprev,
# credit_bureau,
# debitcard,
# deposit,
# other,
# person,
# static,
# static_cb_0,
# tax_registry

In [5]:
# # Merging `train_static`
# train_static = pl.concat(
#     [
#         pl.read_parquet(path + "parquet_files/train/"+ "train_static_0_0.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_static_0_1.parquet")
#     ]
# )

# # Merging `train_applprev`
# train_applprev = pl.concat(
#     [
#         pl.read_parquet(path + "parquet_files/train/"+ "train_applprev_1_0.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_applprev_1_1.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_applprev_2.parquet")
#     ]
# )

# # Merging `train_credit_bureau_a`
# train_credit_bureau_a = pl.concat(
#     [
#         pl.read_parquet(path + "parquet_files/train/"+ "train_credit_bureau_a_1_0.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_credit_bureau_a_1_1.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_credit_bureau_a_2_0.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_credit_bureau_a_2_1.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_credit_bureau_a_2_2.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_credit_bureau_a_2_3.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_credit_bureau_a_2_4.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_credit_bureau_a_2_5.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_credit_bureau_a_2_6.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_credit_bureau_a_2_7.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_credit_bureau_a_2_8.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_credit_bureau_a_2_9.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_credit_bureau_a_2_10.parquet")
#     ]
# )

# # Merging `train_credit_bureau_b`
# train_credit_bureau_b = pl.concat(
#     [
#         pl.read_parquet(path + "parquet_files/train/"+ "train_credit_bureau_b_1.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_credit_bureau_b_2.parquet")
#     ]
# )

# # Merging `train_person`
# train_person = pl.concat(
#     [
#         pl.read_parquet(path + "parquet_files/train/"+ "train_person_1.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_person_2.parquet")
#     ]
# )

# # Loading other single files without concatenation
# train_base = pl.read_parquet(path + "parquet_files/train/"+ "train_base.parquet")
# train_debitcard = pl.read_parquet(path + "parquet_files/train/"+ "train_debitcard_1.parquet")
# train_deposit = pl.read_parquet(path + "parquet_files/train/"+ "train_deposit_1.parquet")
# train_other = pl.read_parquet(path + "parquet_files/train/"+ "train_other_1.parquet")

# # Merging `train_tax_registry`
# train_tax_registry = pl.concat(
#     [
#         pl.read_parquet(path + "parquet_files/train/"+ "train_tax_registry_a_1.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_tax_registry_b_1.parquet"),
#         pl.read_parquet(path + "parquet_files/train/"+ "train_tax_registry_c_1.parquet")
#     ]
# )

# # Merging `train_static_cb`
# train_static_cb = pl.read_parquet(path + "parquet_files/train/"+ "train_static_cb_0.parquet")



In [8]:
train_static_cb = pl.read_parquet(path + "parquet_files/train/"+ "train_static_cb_0.parquet")
train_applprev = pl.concat(
    [
        pl.read_parquet(path + "parquet_files/train/"+ "train_applprev_1_0.parquet"),
        pl.read_parquet(path + "parquet_files/train/"+ "train_applprev_1_1.parquet"),
        pl.read_parquet(path + "parquet_files/train/"+ "train_applprev_2.parquet")
    ]
)

ShapeError: unable to append to a DataFrame of width 41 with a DataFrame of width 6

In [7]:
train_static_cb.head()

case_id,assignmentdate_238D,assignmentdate_4527235D,assignmentdate_4955616D,birthdate_574D,contractssum_5085716L,dateofbirth_337D,dateofbirth_342D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,riskassesment_302T,riskassesment_940T,secondquarter_766L,thirdquarter_1082L
i64,str,str,str,str,f64,str,str,f64,f64,f64,f64,f64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,f64,f64,f64
357,null,null,null,"""1988-04-01""",null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,6.0,6301.4,null,"""2019-01-25""",null,null,null,null,null,null
381,null,null,null,"""1973-11-01""",null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,6.0,4019.6,null,"""2019-01-25""",null,null,null,null,null,null
388,null,null,null,"""1989-04-01""",null,"""1989-04-01""",null,6.0,8.0,2.0,10.0,4.0,"""a55475b1""","""a55475b1""","""a55475b1""",2.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,6.0,"""a55475b1""","""a55475b1""",10.0,null,null,null,null,null,null,6.0,14548.0,null,"""2019-01-28""",null,null,null,null,3.0,5.0
405,null,null,null,"""1974-03-01""",null,"""1974-03-01""",null,0.0,0.0,0.0,1.0,0.0,"""a55475b1""","""a55475b1""","""a55475b1""",0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.0,"""a55475b1""","""a55475b1""",1.0,null,null,null,null,null,null,6.0,10498.24,null,"""2019-01-21""",null,null,null,null,2.0,0.0
409,null,null,null,"""1993-06-01""",null,"""1993-06-01""",null,2.0,3.0,0.0,3.0,1.0,"""a55475b1""","""717ddd49""","""a55475b1""",4.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,"""a7fcb6e5""","""a55475b1""",3.0,null,null,null,null,null,null,7.0,6344.8804,null,"""2019-01-21""",null,null,null,null,0.0,4.0


In [4]:
# def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
#     # implement here all desired dtypes for tables
#     # the following is just an example
#     for col in df.columns:
#         # last letter of column name will help you determine the type
#         if col[-1] in ("P", "A"):
#             df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))

#     return df
# def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
#     for col in df.columns:  
#         if df[col].dtype.name in ['object', 'string']:
#             df[col] = df[col].astype("string").astype('category')
#             current_categories = df[col].cat.categories
#             new_categories = current_categories.to_list() + ["Unknown"]
#             new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
#             df[col] = df[col].astype(new_dtype)
#     return df